In [51]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [52]:
f = open('uspop00.asc','r')

In [53]:
xleft=-180
xright=-65
yup=72
ydown=17
upper_west_point=(xleft,yup)
lower_east_point=(xright,ydown)
latitude_km=40000/360 #it's an approximation as it varies between 111,6 and 110,8 something like that
#for a given latitude, to compute the longitude we need to have the latitude in degrees, and then one degree of
#longitude will be equal to longitude_km=latitude_km*cos(latitude)

In [54]:
linesdf=[]
for line in f:
    linesdf.append(line.split(" "))
df=pd.DataFrame(linesdf)

In [55]:
ncols=int(df[9][0][:-1])
nrows=int(df[9][1][:-1])
xllcorner=int(df[5][2][:-1])
yllcorner=int(df[5][3][:-1])
cellsize=(df[6][4][:-1])
print(ncols,nrows,xllcorner,yllcorner,cellsize)

7080 3120 -125 24 0.0083333333333


In [56]:
df=df.drop([ncols],axis=1)

In [57]:
for i in range(6):
    df=df.drop(df.index[0])

In [58]:
dx=abs(xleft-xright)/ncols
dy=abs(yup-ydown)/nrows
print(dy,dx)

0.017628205128205128 0.016242937853107344


In [59]:
indexes_x=[]
for i in np.arange(xleft,xright,dx):
    indexes_x.append(i)
indexes_y=[]
for j in np.arange(yup,ydown,-dy):
    indexes_y.append(j)
print(len(indexes_x), len(indexes_y))

7080 3120


In [60]:
df.columns=indexes_x
df.index=indexes_y
df

,-180.0,-179.98375706214688,-179.96751412429376,-179.95127118644064,-179.9350282485875,-179.9187853107344,-179.90254237288127,-179.88629943502815,-179.87005649717503,-179.8538135593219,...,-65.16242937843117,-65.14618644057805,-65.12994350272493,-65.1137005648718,-65.09745762701868,-65.08121468916556,-65.06497175131244,-65.04872881345932,-65.0324858756062,-65.01624293775308
72.000000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71.982372,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71.964744,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71.947115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71.929487,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71.911859,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71.894231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71.876603,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71.858974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71.841346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
interval_Lat = []
for i in range(1,len(indexes_y)):
    interval_Lat.append(indexes_y[i]-indexes_y[i-1])

interval_Lng = []
for i in range(1,len(indexes_x)):
    interval_Lng.append(indexes_x[i]-indexes_x[i-1])

# 80x125 (LatxLng) dimensions of image grid and popullation grid of dataset
total_run_Lat = 80*interval_Lat[0]
total_run_Lng =125*interval_Lng[0]
#Hardcoded starting points based on geographical diversity and number of images
start_Lat = 40.27259341025635
start_Lng = -78.36590023164018
end_Lat = start_Lat + total_run_Lat
end_Lng = start_Lng + total_run_Lng

start_Lat, start_Lng, end_Lat, end_Lng, interval_Lat[0], interval_Lng[0]

(40.27259341025635,
 -78.36590023164018,
 38.862336999999997,
 -76.335532999999998,
 -0.017628205128204399,
 0.016242937853121475)

In [62]:
# Filter dataframe for the coordinates selected above
# Note that the above values were calculated over the map, and thus are not neccesarily present in the dataframe
# For this reason, we now capture the exact coordinates available in the dataframe, these are:
real_start_Lat = list(df.index.values)[1800]
real_start_Lng = list(df.columns.values)[6257]
real_end_Lat = list(df.index.values)[1800+79]
real_end_Lng = list(df.columns.values)[6257+124]

real_start_Lat, real_start_Lng, real_end_Lat, real_end_Lng


(40.269230769232081,
 -78.36793785301893,
 38.876602564103933,
 -76.353813559231867)

In [69]:
df_filtered = df.iloc[1800:1800+80,6257:6257+125];
#np.sum(df_filtered.values)
df_filtered.sum().sum()

0.0

In [64]:
import urllib.request
from PIL import Image
import os
import math

class GoogleMapDownloader:
    """
        A class which generates high resolution google maps images given
        a longitude, latitude and zoom level
    """

    def __init__(self, lat, lng, zoom=12):
        """
            GoogleMapDownloader Constructor

            Args:
                lat:    The latitude of the location required
                lng:    The longitude of the location required
                zoom:   The zoom level of the location required, ranges from 0 - 23
                        defaults to 12
        """
        self._lat = lat
        self._lng = lng
        self._zoom = zoom

    def getXY(self):
        """
            Generates an X,Y tile coordinate based on the latitude, longitude 
            and zoom level

            Returns:    An X,Y tile coordinate
        """
        
        tile_size = 256

        # Use a left shift to get the power of 2
        # i.e. a zoom level of 2 will have 2^2 = 4 tiles
        numTiles = 1 << self._zoom

        # Find the x_point given the longitude
        point_x = (tile_size/ 2 + self._lng * tile_size / 360.0) * numTiles // tile_size

        # Convert the latitude to radians and take the sine
        sin_y = math.sin(self._lat * (math.pi / 180.0))

        # Calulate the y coorindate
        point_y = ((tile_size / 2) + 0.5 * math.log((1+sin_y)/(1-sin_y)) * -(tile_size / (2 * math.pi))) * numTiles // tile_size

        return int(point_x), int(point_y)

    def generateImage(self, **kwargs):
        """
            Generates an image by stitching a number of google map tiles together.
            
            Args:
                start_x:        The top-left x-tile coordinate
                start_y:        The top-left y-tile coordinate
                tile_width:     The number of tiles wide the image should be -
                                defaults to 5
                tile_height:    The number of tiles high the image should be -
                                defaults to 5
            Returns:
                A high-resolution Goole Map image.
        """

        start_x = kwargs.get('start_x', None)
        start_y = kwargs.get('start_y', None)
        tile_width = kwargs.get('tile_width', 3)
        tile_height = kwargs.get('tile_height', 5)

        # Check that we have x and y tile coordinates
        if start_x == None or start_y == None :
            start_x, start_y = self.getXY()

        # Determine the size of the image
        width, height = 256 * tile_width, 256 * tile_height

        #Create a new image of the size require
        map_img = Image.new('RGB', (width,height))

        for x in range(0, tile_width):
            for y in range(0, tile_height) :
                url = 'https://mt0.google.com/vt/lyrs=s&x='+str(start_x+x)+'&y='+str(start_y+y)+'&z='+str(self._zoom)
                # Where lyrs=
                # h = roads only
                # m = standard roadmap
                # p = terrain
                # r = somehow altered roadmap
                # s = satellite only
                # t = terrain only
                # y = hybrid
                #print(url)
                current_tile = str(x)+'-'+str(y)
                urllib.request.urlretrieve(url, current_tile)
            
                im = Image.open(current_tile)
                map_img.paste(im, (x*256, y*256))
              
                os.remove(current_tile)

        return map_img

In [68]:
def main():
    # Create a new instance of GoogleMap Downloader
    for Lat in range(80):
        for Lng in range(125):
            gmd = GoogleMapDownloader(real_start_Lat + (Lat*interval_Lat[0]), real_start_Lng + (Lng*interval_Lng[0]), 16)
            #print("The tile coorindates are {}".format(gmd.getXY()))

            try:
                # Get the high resolution image
                img = gmd.generateImage()
                img = img.resize((192, 320), Image.ANTIALIAS)
            except IOError:
                print("Could not generate the image - try adjusting the zoom level and checking your coordinates")
            else:
                #Save the image to disk
                img.save(str(Lat)+"_"+str(Lng)+".png")
                #print("The map has successfully been created in the loop: " + str(i))


if __name__ == '__main__':  main()